In [1]:
import random
import numpy as np
import scipy.stats
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('evanrossgreen', 'YyptFQgf4L2z61PrGDKS')
#py.sign_in('glen_weyl270', 'LxmiimP0wmLyrTcaJhjA')
#py.sign_in('glen_weyl271', 'pPXVCkG0eE3kLwB8Cbbg')
#py.sign_in('glen_weyl272', 'glen_weyl272')
#py.sign_in('glen_weyl273', 'glen_weyl273')
#py.sign_in('glen_weyl274', 'glen_weyl274')
#py.sign_in('glen_weyl275', 'XSdUnS8x4HSDRuIVru4i')

#Simulation for DDE Final                                                                                                   
#random thought - this is similar to the internet usage problem except 
#i think it's totally fine to declare prices at the current moment when they make a decision, unlike internet pricing which is confusing

#DONE
#----------------------------------------------------------------------------------------------------------------------------------------

#Model mph based on cars DONE
#Model mode choice DONE
    #Add a utility function for taking a bus DONE
    #Make it so that bus is an option DONE
    #Add in abilities to deal with bus people (fractional people?) DONE
    #I think we will assume that people who bus in will bus out DONE
#Land use
    #Keep track of how many cars are parked at each time DONE    
#Create some framework for calculating overall utility DONE
    #Keep track of what choices people are making DONE
    #Calculate total cost of utility of the drivers from the choice (park, circle, leave, bus) DONE
        #Calculate how this utility changes for varying taxes based on where you are in the income distribution DONE
    #Calculate total gain of utility from the land being freed up DONE
    #Calculate total tax levied DONE
#Add in the cost for everything when entering the city (do we want some model based on how far away people are?)    
#Add penetration model


#TODO
#----------------------------------------------------------------------------------------------------------------------------------------
#Compute total miles traveled
#Create some framework calculating equilibrium time of travel
    #write move_trip(num_minutes)

# SEE CONGESTION MODELLING PAPER
'''http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.17.7708&rep=rep1&type=pdf'''




'http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.17.7708&rep=rep1&type=pdf'

In [11]:
def simulate(penetration = 1, driving_tax_per_hour = 0, parking_tax_per_hour = 0):
    ###set up some constants for use later so we can easily change things 

    k_total_time = 24 * 60 * 2
    k_interval_length = 2
    k_time_periods = k_total_time/k_interval_length #because otherwise you get out of range because cars in the 24th hour still come in
    k_hour_length  = 60 
    k_workday_length = 8  

    #https://ag.tennessee.edu/cpa/Information%20Sheets/CPA%20222.pdf
    parking_to_land_ratio = 180 #you need parking_to_land_ratio square feet to park a car 

    #This number is kind of bullshit, but I found a business insider article which said you can buy 2000 sqft of real estate in boston for 1 mil
    #obvs real estate is not equiv to just land, but whatever it's easy to change
    land_to_utility_ratio = .5 #a city would be willing to pay land_to_utility_ratio dollars for a square foot of land for a day
    #variables to be modified, but stay constant throughout the simulation
    city_radius = 10
    driving_cost_per_hour = 1 #experimenting to get some people to circle
    parking_cost_per_hour = 7.5 # I just changed this, to make it more accurate, 60 dollars a day
    #parking_tax_per_hour = 0 #this should be per unit (no, just make it parking_tax_per_hour - if you want to make it per unit, do that when choosing its value)

    parking_outside_cost_per_hour = 1.25 #less than in city 
    bus_fare = 2.5 * 2 #there and back 
    bus_extra_time = 2 # in hours
    bus_capacity = 20 #if bus_capacity people decide to bus to work, it means that there is the equivalent of one additional car on the road

    #variables which should be modified at each time step of the simulation
    #time between telling the car to park and it parking
    parking_waiting_time = 10 # should be based on schoup
    #time between telling a circling car to get you and it arriving
    circling_waiting_time = 5
    mph_in_city = 20
    leaving_waiting_time = 15 #time between telling a car to leave the city and it leaving
    leaving_time = 8 #there is probably some uncertainty here because it could get stuck in traffi
    num_cars_parked_in_city = 0    
    num_cars_driving_in_city = 0

    #Put a car in parked_in_city[i] if at time i * k_len_interval it won't be parked anymore
    parked_in_city = [0] * k_time_periods
    #same idea
    driving_in_city = [0] * k_time_periods
    #Put a car in to_be_parked_in_city[i] if in (i+1)15 minutes it will be parked(you might put it here right after somebody decides to park their car, for instance)
    #command is:
    #to_be_parked_in_city[4].append(400) if you want to park a car for 400 minutes an hour from now


    #List of variables we are interested in studying
    study_parked_in_city = []
    study_driving_in_city = []

    #Keep track of what choices people are making 
    study_choices = {} #keys are wages, values are a list of choices (0, 1, 2, 3)

    #Calculate total cost of utility of the drivers from the choice (park, circle, leave, bus)
    study_utility_drivers = []

    #Keep track of the changed utility of people depending on their wage
    study_utility_by_wage = {} #keys are wages, values are a list of costs

    study_driving_tax_levied = []
    study_parking_tax_levied = []

    study_miles_driven = []

    study_leaving_time = []

    study_utility_leaving_parkers = []
    study_utility_leaving_circlers = []
    study_utility_leaving_leavers = []
    study_utility_leaving_bussers = []

    #End list
    #vmt is just a sum
    study_VMT = 0
    study_overall_utility = 0
    
    parking_average_wage = {}
    circling_average_wage = {}
    leaving_average_wage = {}
    bussing_average_wage ={}

    def get_bus_utility(wage):
        return bus_fare + bus_extra_time * wage + leaving_waiting_time/60.0 * wage
        #cost to use the bus 

    def get_parking_utility(wage):       
        waiting_cost = wage * (parking_waiting_time/60.0)
        actual_cost = k_workday_length * (parking_cost_per_hour + parking_tax_per_hour)
        return waiting_cost + actual_cost

    def get_circle_utility(wage):
        waiting_cost = wage * (circling_waiting_time/60.0)
        actual_cost = k_workday_length * (driving_tax_per_hour + driving_cost_per_hour)
        return waiting_cost + actual_cost

    def get_leave_utility(wage, time_driving = 20):
        waiting_cost = wage * (leaving_waiting_time/60.0)
        #should there be a 2 here???
        leaving_cost = 2 * (leaving_time/60.0) * (driving_tax_per_hour + driving_cost_per_hour)
        driving_cost = 2 * (time_driving/60.0) * driving_cost_per_hour
        parking_cost = (k_workday_length - 2.0/60.0 * leaving_time - 2.0/60.0 * time_driving) * parking_outside_cost_per_hour
        return waiting_cost + leaving_cost + driving_cost + parking_cost

    to_be_parked_in_city = {}
    for i in range(k_time_periods):
        to_be_parked_in_city[i] = []
        parking_average_wage[i] = (0,0)
        circling_average_wage[i] = (0,0)
        leaving_average_wage[i] = (0,0)
        bussing_average_wage[i] = (0,0)
    #same idea
    to_be_driving_in_city = {}
    for i in range(k_time_periods):
        to_be_driving_in_city[i] = []

    def get_mph(cars_driving_in_city):
        max_mph = 40
        min_mph = 1
        if cars_driving_in_city < 1000:
            return max_mph
        else:
            return max(max_mph - cars_driving_in_city ** 1.5/ (10 ** 6.5),min_mph)

    k_max_arrival_time = 9
    interval_pdf = 0
    for dummy_time in range(0, k_total_time, k_interval_length):
        interval_pdf += scipy.stats.norm(k_max_arrival_time * k_hour_length, k_hour_length).pdf(dummy_time)

    total_people = 10 ** 5
    
    def get_num_people_arriving_in_city(time):
        total_adjusted_people = total_people / interval_pdf
        return int((scipy.stats.norm(k_max_arrival_time * k_hour_length, k_hour_length).pdf(time) * total_adjusted_people))


    which = lambda lst:list(np.where(lst)[0])

    def get_wage(): 
        percentile = random.randint(1,100)
        census = [0,6,9,12,17,22,26,31,34,39,42,46,48,53,55,58,60,63,65,67,69,72,73,
                    75,76,78,79,80,81,83,83,85,85,86,87,88,88,89,89,90,90,100]
        income = 0
        while income == 0: 
            if percentile in census:
                income = np.mean(which([a == percentile for a in census]))
            else: 
                percentile += 1
        #adjsut the top end of the distribution
        if income == 100:
            income += random.randint(1,50) + random.randint(1,50)
        return (income * 2500 - 1250) / 2000  #work 2000 hours in a year 

    #returns 0 = park, 1 = circle, 2 = leave, 3 = bus
    def choose(wage, self_driving):
        if self_driving:
            results = [get_parking_utility(wage), get_circle_utility(wage), get_leave_utility(wage), get_bus_utility(wage)]
        else:
            results = [get_parking_utility(wage), 10000000000, 10000000000, get_bus_utility(wage)]
        return (results.index(min(results)), min(results))

    #Evan added that revenue gained isn't lost since we return it to consumers
    def calculate_overall_utility():
        total_utility = 0
        personal_cost = sum(study_utility_drivers) #The utility cost people are paying for their transportation to the city
        total_utility -= personal_cost
        total_utility -= sum(study_utility_leaving_parkers)
        total_utility -= sum(study_utility_leaving_circlers)
        total_utility -= sum(study_utility_leaving_leavers)
        total_utility -= sum(study_utility_leaving_bussers)
        land_cost = max(study_parked_in_city) * parking_to_land_ratio * land_to_utility_ratio
        total_utility -= land_cost
        tax_revenue = sum(study_driving_tax_levied) + sum(study_parking_tax_levied)
        total_utility += tax_revenue

        return total_utility

    for time in range(0, k_total_time/2, k_interval_length): 
        arrivals = get_num_people_arriving_in_city(time)
        #deal with new arrivals
        sum_personal_costs = 0
        sum_parking_taxes = 0
        sum_driving_taxes = 0
        sum_miles_driven = 0
        for i in range(arrivals):
            wage = get_wage() #this should be randomly generated according to our distribution eventually
            #This is the amount of time the car is actually going to spend parked
            time_until_next_use = 8 * 60 #this should also be randomly generated according to some distribution eventually
            #gotta be a little careful in case time until next use is less than leaving/parking times(the decision shouldn't be to leave/park then)
            #Check if we are self driving or not
            self_driving = False
            if (np.random.rand(1)[0] <= penetration):
                self_driving = True
            (decision, personal_cost) = choose(wage, self_driving)
            sum_personal_costs += personal_cost
            #0 = park, 1 = circle, 2 = leave, 3 = bus
            if (int(wage) not in study_choices.keys()):
                study_choices[int(wage)] = []
            study_choices[int(wage)].append(decision)
            if (int(wage) not in study_utility_by_wage.keys()):
                study_utility_by_wage[int(wage)] = []
            study_utility_by_wage[int(wage)].append(personal_cost)
            if decision == 0: #park
                #Drive into the city to your destination (at time: time)
                #Tell your car to park (at time: time + leaving_time)
                to_be_parked_in_city[(parking_waiting_time + leaving_time + time)/k_interval_length].append(time_until_next_use-parking_waiting_time)
                driving_in_city[(parking_waiting_time + leaving_time + time)/k_interval_length + 1] += 1

                #keep the average wage and count of people parking
                index = (time_until_next_use + leaving_time + time)/k_interval_length
                first_elt = (parking_average_wage[index][0]*parking_average_wage[index][1]+wage)/(parking_average_wage[index][1]+1)
                second_elt = parking_average_wage[index][1]+1
                parking_average_wage[index] = (first_elt, second_elt)

                #Compute taxes for parking and driving 
                sum_parking_taxes += 1.0/60.0 * (time_until_next_use - parking_waiting_time) * parking_tax_per_hour
                sum_driving_taxes += (leaving_time + parking_waiting_time)/60.0 * driving_tax_per_hour
            elif decision  == 1: #circle
                #Drive your car into the city, where it drops you off
                #simulate circling a car
                sum_driving_taxes += (leaving_time + time_until_next_use)/60.0 * driving_tax_per_hour
                driving_in_city[(time + time_until_next_use + leaving_time)/k_interval_length + 1]+=1

                index = (time + time_until_next_use + leaving_time)/k_interval_length + 1
                first_elt = (circling_average_wage[index][0]*circling_average_wage[index][1]+wage)/(circling_average_wage[index][1]+1)
                second_elt = circling_average_wage[index][1]+1
                circling_average_wage[index] = (first_elt,second_elt)

                to_be_driving_in_city[(time + time_until_next_use + leaving_time)/k_interval_length + 1].append(-1) #you still have to find your owner and then leave

            elif decision == 2: #leave
                sum_driving_taxes += (2*leaving_time)/60.0 * driving_tax_per_hour
                #simulate going in and leaving the city
                driving_in_city[(time + 2*leaving_time)/k_interval_length + 1] += 1
                #when you are outside the city nobody cares about you
                #you will come into the city
                to_be_driving_in_city[(time_until_next_use + leaving_time + time)/k_interval_length + 1].append(-3)

                index = (time_until_next_use + leaving_time + time)/k_interval_length + 1
                first_elt = (leaving_average_wage[index][0]*leaving_average_wage[index][1]+wage)/(leaving_average_wage[index][1]+1)
                second_elt = leaving_average_wage[index][1]+1
                leaving_average_wage[index] = (first_elt,second_elt)
            elif decision == 3: #bus
                #Don't tax the bus
                #The bus enters and leaves the city
                driving_in_city[(time + 2*leaving_time)/k_interval_length + 1] += 1.0/bus_capacity
                #You have to leave the city on a bus at some point
                to_be_driving_in_city[(leaving_time + time_until_next_use + int(60.0 * bus_extra_time) + time)/k_interval_length + 1].append(-2)
                index = (int(time_until_next_use + int(60.0 * bus_extra_time)/2)/k_interval_length)
                first_elt = (bussing_average_wage[index][0]*bussing_average_wage[index][1]+wage)/(bussing_average_wage[index][1]+1)
                second_elt = bussing_average_wage[index][1]+1
                bussing_average_wage[index] = (first_elt,second_elt)
        #update all the variables which need to be updated

        #deal with the driving people who are leaving

        #deal with the parked people who are leaving
        leaving_from_park = parked_in_city[time/k_interval_length]
        avg_park_wage =  parking_average_wage[time/k_interval_length]
        utility_lost = avg_park_wage[0] * (leaving_time/60.00) * leaving_from_park
        #print ("Utility Lost by parkers because of driving time, leaving at time " + str(time) + ": " + str(utility_lost))
        #They all get taxed for driving
        sum_driving_taxes += leaving_from_park * leaving_time/60.0 * driving_tax_per_hour
        utility_lost += leaving_from_park * leaving_time/60.0 * driving_tax_per_hour
        study_utility_leaving_parkers.append(utility_lost)

        #Update drivers in city
        driving_in_city[(time + leaving_time)/k_interval_length + 1] += leaving_from_park

        #deal with the cars which are about to park 
        about_to_park = to_be_parked_in_city[time/k_interval_length]
        for car in about_to_park:
            parked_in_city[(car+time)/k_interval_length + 1] += 1

        #deal with the cars which are about to drive
        about_to_drive = to_be_driving_in_city[time/k_interval_length]
        utility_lost_leavers_i = 0
        utility_lost_circlers_i = 0
        utility_lost_busers_i = 0
        for car in about_to_drive:
            if car == -1: #leavers
                avg_leave_wage =  leaving_average_wage[time/k_interval_length]
                tax_i = leaving_time/60.0 * driving_tax_per_hour
                utility_lost = avg_leave_wage[0] * (leaving_time/60.00) + tax_i
                utility_lost_leavers_i += utility_lost
                sum_driving_taxes += tax_i
                #These are cars that were outside the city and are coming back, so one leaving_time is for going in and the other leaving time is for going back out
                driving_in_city[(time + leaving_time)/k_interval_length] += 1 
            elif car == -2: #bussers
                avg_bus_wage = bussing_average_wage[time/k_interval_length]
                utility_lost = (leaving_time/60.0 + bus_extra_time) * avg_bus_wage[0]
                utility_lost_busers_i += utility_lost
                driving_in_city[(time + leaving_time)/k_interval_length] += 1.0/bus_capacity
            elif car == -3: #circlers
                avg_circle_wage = circling_average_wage[time/k_interval_length]
                tax_i = 1.5 * leaving_time/60.0 * driving_tax_per_hour
                utility_lost = (leaving_time/60.0 + circling_waiting_time/60.0) * avg_circle_wage[0] + tax_i
                sum_driving_taxes += tax_i
                utility_lost_circlers_i += utility_lost
                driving_in_city[int(time + leaving_time/k_interval_length)] += 1
            else:
                sum_driving_taxes += car/60.0 * driving_tax_per_hour
                driving_in_city[car/k_interval_length] += 1
        study_utility_leaving_circlers.append(utility_lost_circlers_i)
        study_utility_leaving_leavers.append(utility_lost_leavers_i)
        study_utility_leaving_bussers.append(utility_lost_busers_i)    

        #update variables we want to keep track of
        #should these be arrays or just static values?
        num_cars_driving_in_city = sum(driving_in_city) - sum(driving_in_city[0:time/k_interval_length])
        num_cars_parked_in_city = sum(parked_in_city) - sum(parked_in_city[0:time/k_interval_length]) 

        mph_in_city = get_mph(num_cars_driving_in_city)
        average_leaving_distance = city_radius/3.0 #computed using an integral, math230 paid off!!!
        leaving_time = int(average_leaving_distance/mph_in_city * 60.0)
        parking_waiting_time = 10
        circling_waiting_time = 5
        leaving_waiting_time = max(leaving_time/10.0,5)

        study_parked_in_city.append(num_cars_parked_in_city)
        study_driving_in_city.append(num_cars_driving_in_city)
        study_utility_drivers.append(sum_personal_costs)
        study_parking_tax_levied.append(sum_parking_taxes)
        study_driving_tax_levied.append(sum_driving_taxes)
        study_leaving_time.append(leaving_time)

        #study vehicle miles travled
        study_VMT += mph_in_city * num_cars_driving_in_city * k_interval_length / 60 

        #if time % 1000 == 0:
        #    print ("time: " + str(time))
        #    print num_cars_driving_in_city

    study_dict = {}
    study_dict['study_parked_in_city'] = study_parked_in_city
    study_dict['study_driving_in_city'] =study_driving_in_city
    study_dict['study_choices']= study_choices
    study_dict['study_utility_drivers']=study_utility_drivers
    study_dict['study_utility_by_wage']=study_utility_by_wage
    study_dict['study_driving_tax_levied']=study_driving_tax_levied
    study_dict['study_parking_tax_levied']=study_parking_tax_levied
    study_dict['study_miles_driven']=study_miles_driven
    study_dict['study_leaving_time']=study_leaving_time
    study_dict['study_utility_leaving_parkers']=study_utility_leaving_parkers
    study_dict['study_utility_leaving_circlers']=study_utility_leaving_circlers
    study_dict['study_utility_leaving_leavers']=study_utility_leaving_leavers
    study_dict['study_utility_leaving_bussers']=study_utility_leaving_bussers
    study_dict['study_VMT']=study_VMT
    study_dict['study_overall_utility'] = calculate_overall_utility()
    return study_dict

In [12]:
#Study the penetration rate
num_elts = 6
utility_by_penetration_rate = []
percent_parking = []
percent_circling = []
percent_leaving = []
percent_bussing = []
VMT = []
leaving_time = []
for dummy in range(0,num_elts):
    if dummy % 5 == 0:
        print (dummy)
    penetration = (100 /(num_elts-1)) * dummy/100.0
    study_dict = simulate(penetration = penetration)
    utility_by_penetration_rate.append(study_dict['study_overall_utility'])
    study_choices = study_dict['study_choices']
    count_study_choices = [0] * 4
    for key in study_choices.keys():
        for val in study_choices[key]:
            count_study_choices[val] += 1
    proportions = np.round(np.divide(np.array(count_study_choices),sum(count_study_choices)+0.0),3)
    percent_parking.append( proportions[0])
    percent_circling.append(proportions[1])
    percent_leaving.append( proportions[2])
    percent_bussing.append(proportions[3])
    VMT.append(study_dict['study_VMT'])

0
5


In [28]:
#Study the tax rate
num_elts = 10
utility_by_tax_rate = []
percent_parking = []
percent_circling = []
percent_leaving = []
percent_bussing = []
VMT = []
max_tax = 20.0
for dummy in range(0,num_elts):
    if dummy % 5 == 0:
        print dummy
    driving_tax_per_hour = dummy * max_tax / num_elts
    study_dict = simulate(driving_tax_per_hour = driving_tax_per_hour)
    utility_by_tax_rate.append(study_dict['study_overall_utility'])
    study_choices = study_dict['study_choices']
    count_study_choices = [0] * 4
    for key in study_choices.keys():
        for val in study_choices[key]:
            count_study_choices[val] += 1
    proportions = np.round(np.divide(np.array(count_study_choices),sum(count_study_choices)+0.0),3)
    percent_parking.append( proportions[0])
    percent_circling.append(proportions[1])
    percent_leaving.append( proportions[2])
    percent_bussing.append(proportions[3])
    VMT.append(study_dict['study_VMT'])

0
5


In [29]:
study_dict['study_leaving_time']

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,


In [5]:
columns = ['park', 'circle', 'leave', 'bus']

In [13]:
###make bar plot of choices ####

data = [go.Bar(
            x=columns,
            y=proportions
    )]

layout = go.Layout(
    title='Proportion of Commuters Choosing Each Transportation Option',
    xaxis=dict(
        title='',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Percent',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        range=[0, 1]
    )
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')


In [14]:
#### Study Total Utility as Proprotion of Drivers with Self Driving Cars Increases, No Taxes

trace0 = go.Scatter(
    x = np.linspace(0,1,num_elts),
    y = utility_by_penetration_rate,
    mode = 'lines+markers'
)
layout = go.Layout(
    title='Utility as Number of AVs increase',
    xaxis=dict(
        title='Penetration Rate',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Utility',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
    )
)
data = [trace0]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')

In [15]:
### Study how proportions of commuters doing each of the different things changes ### 
trace0 = go.Scatter(
    x = np.linspace(0,1,num_elts),
    y = percent_parking,
    mode = 'lines+markers',
    name = 'Park'
)
trace1 = go.Scatter(
    x = np.linspace(0,1,num_elts),
    y = percent_circling,
    mode = 'lines+markers',
    name = 'Circling'
)
trace2 = go.Scatter(
    x = np.linspace(0,1,num_elts),
    y = percent_leaving,
    mode = 'lines+markers',
    name = 'Leaving',
)
trace3 = go.Scatter(
    x = np.linspace(0,1,num_elts),
    y = percent_bussing,
    mode = 'lines+markers',
    name = 'Bussing',
)
layout = go.Layout(
    title='Commuting Decisions as Penetration Rate Increases',
    xaxis=dict(
        title='Penetration Rate',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Proportion',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        range = [0,1]
    )
)
data = [trace0,trace1,trace2,trace3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Proportion-line-mode')

In [34]:
#### Study Total Utility as driving taxes increases, penetration rate is 1 

trace0 = go.Scatter(
    x = np.linspace(0,max_tax,num_elts),
    y = utility_by_tax_rate,
    mode = 'lines+markers'
)
layout = go.Layout(
    title='Utility and Driving Taxes',
    xaxis=dict(
        title='Tax Rate on Driving',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Utility',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
    )
)
data = [trace0]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')

In [35]:
### Study how proportions of commuters doing each of the different things when we change the tax rate 
x_range = np.linspace(0,max_tax,num_elts)
trace0 = go.Scatter(
    x = x_range,
    y = percent_parking,
    mode = 'lines+markers',
    name = 'Park'
)
trace1 = go.Scatter(
    x = x_range,
    y = percent_circling,
    mode = 'lines+markers',
    name = 'Circling'
)
trace2 = go.Scatter(
    x = x_range,
    y = percent_leaving,
    mode = 'lines+markers',
    name = 'Leaving',
)
trace3 = go.Scatter(
    x = x_range,
    y = percent_bussing,
    mode = 'lines+markers',
    name = 'Bussing',
)
layout = go.Layout(
    title='Drivers Choices as Tax Rate Increases',
    xaxis=dict(
        title='Tax Rate',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Proportion',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        range = [0,1]
    )
)
data = [trace0,trace1,trace2,trace3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Proportion-line-mode')

In [40]:
#### Study VMT

trace0 = go.Scatter(
    x = np.linspace(0,max_tax,num_elts),
    y = VMT,
    mode = 'lines+markers'
)
layout = go.Layout(
    title='VMT',
    xaxis=dict(
        title='Tax Rate',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='VMT',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
    )
)
data = [trace0]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')